In [7]:
import simpy
import numpy as np
import random

In [3]:
def suspicion(pre_suspicion, alpha, baseline, outcome, expectation) -> float:
    '''Returns suspicion level according to hypothesized computational behaviour model'''
    return pre_suspicion + alpha * (outcome - expectation) + baseline

In [16]:
class Trial(object):
    n_cards = 5
    cards = []
    
    def __init__(self, n_red, outcome):
        ''' blue cards always win, red cards always lose, hence values 1 and -1 respectively '''
        self.n_red = n_red
        self.n_blue = n_cards - n_red
        cards.extend([-1] * n_red)
        cards.extend([1] * n_blue)
        self.outcome = outcome
    
    def expectation(self):
        if self.outcome == -1:
            return self.outcome * self.n_red / n_cards
        if self.outcome == 1:
            return self.outcome * self.n_blue / n_cards
        else:
            raise ValueError("Opponent outcome has unacceptable value.")
    
    def playerSelection(self):
        return random.choice(cards)

In [ ]:
class Player(object):
    def __init__(self, alpha, baseline, pre_suspicion):
        self.alpha = alpha
        self.baseline = baseline
        if pre_suspicion is None:
            self.presuspicion = 0
        self.pre_suspicion = pre_suspicion
        
    def notLie(player_selection):
        # expects input from trial.playerSelection()
        return player_selection
    
    def lie(player_selection):
        #todo: expect higher probability that Player will lie if suspicion is high
        if player_selection == -1:
            return 1

In [8]:
# game play values
n_trials = 100
n_red = range(0, 5, 1)
outcome = (1, -1)

# player values
pre_suspicion = 0
alpha = np.linspace(0,1pa,11)






params = {'alpha': alpha, ''}

for p in params:
    trial(p)
    suspicion(p)

In [9]:
alpha

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])